## Functionality for calculating token balances 

In [1]:
# import dependencies 
import time
from requests import get, post
import requests

### Extract data from Dune API

In [2]:
def make_api_url(module, action, ID, base_url):
    """
    Generate a URL to call the API.
    """

    url = base_url + module + "/" + ID + "/" + action

    return url

def execute_query(api_url, headers, engine):
    """
    Calls the API to execute the query.
    :param: api_url: the api_url of the query.
    :param: engine: engine size will change how quickly the query runs.   
    :returns: the execution ID of the instance which is executing the query.
    """
    params = {
        "performance": engine,
    }
    response = post(url=api_url, headers=headers, params=params)
    execution_id = response.json()['execution_id']

    return execution_id

def get_query_status(status_url, headers):
    """
    Fetches the status of query execution using the API
    :param: execution id: execution id of the query.
    :returns: returns the response object.
    """
    response = get(url=status_url, headers=headers)

    return response

def get_query_results(results_url, headers):
    """
    Fetches the results returned from the query using the API.
    :param: execution id: execution id of the query.
    :returns: returns the response object.
    """
    response = get(url=results_url, headers=headers)

    return response

def cancel_query_execution(cancel_url, headers):
    """
    Cancels the ongoing execution of the query.
    :param: execution id: execution id of the query.
    :resturns: returns the response object.
    """
    response = get(url=cancel_url, headers=headers)

    return response

def wait_for_query_completion(status_url, headers, interval=5, timeout=600):
    """
    Waits for a query to complete and prints the final state and returns the results.
    :param: execution_id: str, the ID of the query execution.
    :param: interval (int): the interval (in seconds) between status checks.
    :returns: final response status.
    """
    start_time = time.time()

    # Initial request to get the query status
    response_status = get_query_status(status_url, headers)
    state = response_status.json()['state']

    # Continue checking the state until it becomes 'QUERY_STATE_COMPLETED'
    while state == 'QUERY_STATE_EXECUTING' and (time.time() - start_time) < timeout:
        time.sleep(interval)
        response_status = get_query_status(status_url, headers)
        state = response_status.json()['state']

    print("Final state:", state)
    
    return response_status.json()

### Query hopr node and calculate channel balances 

In [3]:
def get_unique_nodeAddress_peerId_aggbalance_links(api_host_hopr_node, api_key_hopr_node):
    """
    Returns a dict containing all unique source_peerId-source_address links.
    """
    channel_url = "http://{}:3001/api/v3/channels/?includingClosed=false&fullTopology=true".format(api_host_hopr_node)
    headers = {'X-Auth-Token': api_key_hopr_node}
    response = requests.request("GET", channel_url, headers=headers)

    if response.status_code != 200:
        print("Could not fetch channel information. Status code: {}".format(response.status_code))
        return {}

    response = response.json()

    if 'all' not in response:
            print("Response does not contain `all`")
            return {}

    peerid_address_aggbalance_links = {}
    for item in response["all"]:
        if "sourcePeerId" not in item or "sourceAddress" not in item:
            print("Response does not contain `source_peerid` or `source_address`")
            continue

        if "status" not in item:
            print("Response does not contain `status`")
            continue

        source_peer_id = item["sourcePeerId"]
        source_address = item["sourceAddress"]
        balance = int(item["balance"]) / 1e18

        if item["status"] != "Open":
            # Other Statuses: "Waiting for commitment", "Closed", "Pending to close"
            continue

        if source_peer_id not in peerid_address_aggbalance_links:
            peerid_address_aggbalance_links[source_peer_id] = {
                "source_node_address": source_address,
                "channels_balance": balance,
            }

        else:
            peerid_address_aggbalance_links[source_peer_id][
                "channels_balance"
            ] += balance

    return peerid_address_aggbalance_links